1. Fit DesicisionTree and Predict with Train Validate Data Split 


In [1]:
import re
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
data_path_raw = Path.cwd().parent.parent / "data" / "raw"

data_path_preprocessed = Path.cwd().parent.parent / "data" / "processed"


In [3]:
raw_df_processed = pd.read_csv(data_path_preprocessed / "training_set_features__Preprocessing_nominal_ordinal_WOE_Impute.csv", index_col="respondent_id")
labels_df = pd.read_csv(data_path_raw / "training_set_labels.csv", index_col="respondent_id")
test_df_processed = pd.read_csv(data_path_preprocessed / "test_set_features__Preprocessing_nominal_ordinal_WOE_Impute.csv", index_col="respondent_id")

all_raw_df = raw_df_processed.join(labels_df)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    raw_df_processed,
    labels_df,
    shuffle = True,
    test_size = 0.25,
    random_state = 10)

In [57]:
from sklearn.tree import DecisionTreeClassifier

import time 

start = time.time()
dec_tree = DecisionTreeClassifier()
dec_tree.fit(X_train, y_train)
test_predictions = dec_tree.predict(X_test)
test_probability = dec_tree.predict_proba(X_test)
end = time.time()
print(end - start)




0.23893094062805176


In [58]:
y_preds = pd.DataFrame(
    {
        "h1n1_vaccine": test_probability[0][:, 1],
        "seasonal_vaccine": test_probability[1][:, 1],
    },
    index = y_test.index
)
print("y_preds.shape:", y_preds.shape)
y_preds.head()

y_preds.shape: (6677, 2)


,h1n1_vaccine,seasonal_vaccine
respondent_id,,
2653,0.0,0.0
9506,0.0,1.0
23107,0.0,1.0
22648,0.0,1.0
25589,1.0,1.0


In [59]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_preds))

0.6718059327263535


In [49]:
dec_tree.get_depth()

35

In [50]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(dec_tree.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False)
importances

,feature,importance
19,opinion_seas_risk,0.117
9,doctor_recc_h1n1,0.061
10,doctor_recc_seasonal,0.057
18,opinion_seas_vacc_effective,0.055
21,age_group,0.047
...,...,...
62,employment_industry_phxvnwax,0.000
31,sex_nan,0.000
51,census_msa_nan,0.000
29,race_nan,0.000


In [53]:
from sklearn import tree

tree.plot_tree(dec_tree)

[Text(198.06334896009668, 214.42, 'X[19] <= 2.86\ngini = 0.416\nsamples = 20030\nvalue = [[15766, 4264]\n[10732, 9298]]'),
 Text(121.74698974139832, 208.38, 'X[10] <= 0.665\ngini = 0.328\nsamples = 11564\nvalue = [[10035, 1529]\n[7989, 3575]]'),
 Text(92.65639815564847, 202.34, 'X[18] <= 4.013\ngini = 0.27\nsamples = 9046\nvalue = [[8095, 951]\n[6978, 2068]]'),
 Text(68.2721546641526, 196.3, 'X[21] <= 3.5\ngini = 0.211\nsamples = 6801\nvalue = [[6237, 564]\n[5712, 1089]]'),
 Text(55.67088168395993, 190.26, 'X[9] <= 0.61\ngini = 0.185\nsamples = 5619\nvalue = [[5173, 446]\n[4895, 724]]'),
 Text(43.62522599840615, 184.22, 'X[73] <= 0.852\ngini = 0.175\nsamples = 5419\nvalue = [[5054, 365]\n[4719, 700]]'),
 Text(25.03104095254684, 178.18, 'X[19] <= 1.5\ngini = 0.161\nsamples = 5203\nvalue = [[4891, 312]\n[4586, 617]]'),
 Text(15.201214627350502, 172.14, 'X[16] <= 2.671\ngini = 0.1\nsamples = 2217\nvalue = [[2125, 92]\n[2075, 142]]'),
 Text(12.030626851845104, 166.1, 'X[73] <= 0.387\ngini 

Error in callback <function flush_figures at 0x7ffbe8810440> (for post_execute):


KeyboardInterrupt: 

In [78]:
from sklearn.metrics import classification_report
#print(classification_report(y_train, train_predictions))
print(classification_report(y_test, test_predictions))

              precision    recall  f1-score   support

           0       0.47      0.48      0.47      1410
           1       0.66      0.65      0.66      3137

   micro avg       0.60      0.60      0.60      4547
   macro avg       0.56      0.57      0.56      4547
weighted avg       0.60      0.60      0.60      4547
 samples avg       0.30      0.30      0.29      4547



/Users/johannes/opt/anaconda3/envs/py37-ds/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johannes/opt/anaconda3/envs/py37-ds/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
